2) We're going to take a closer look at the hanging cable (catenary) problem

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/Documents/Courses/Robot Dynamics 2021/hw2/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Plots
using OrdinaryDiffEq
plotlyjs()

WebIO._IJuliaInit()

Plots.PlotlyJSBackend()

In [3]:
#Problem parameters (feel free to play with these)
g = 9.81
w = 10 #width of gap
ℓ = 15 #length of cable
m = 10 #mass of cable
n = 25 #number of discrete segments

Δx = w/n #width of each discrete segment
ρ = m/ℓ; #mass density

In [4]:
#Total cable length
function S(y,ẏ)
    s = 0.0
    
    ### Solution
    for k = 1:(n-1)
        s += sqrt(1 + ẏ[k]^2)*Δx
    end
    return s
end

function S(y)
    ẏ = (y[2:end]-y[1:(end-1)])./Δx
    S(y,ẏ)
end

S (generic function with 2 methods)

In [5]:
#Potential energy
function U(y,ẏ)
    u = 0.0
    
    ### Solution
    for k = 1:(n-1)
        ym = 0.5*(y[k+1]+y[k])
        u += ρ*g*ym*sqrt(1 + ẏ[k]^2)*Δx
    end
    return u
end

function U(y)
    ẏ = (y[2:end]-y[1:(end-1)])./Δx
    U(y,ẏ)
end

U (generic function with 2 methods)

In [6]:
#End points (feel free to play with these)
y1 = 0.0
yn = 0.0

0.0

In [7]:
#Objective and constraint functions for IPOPT

function objective(z)
    y = z[1:n]
    ẏ = z[(n+1):end]
    
    U(y,ẏ) #minimize potential energy
end

function constraint!(c,z)
    y = z[1:n]
    ẏ = z[(n+1):end]
    
    c .= [
        0.0 #initial point
        0.0 #final point
        0.0 #length constraint
        zeros(length(ẏ)) #velocity constraints
    ]
    
    ### Solution
    c .= [
        y[1] - y1 #initial point
        y[end] - yn #final point
        S(y,ẏ) - ℓ #length constraint
        y[2:end] .- y[1:(end-1)] .- ẏ.*Δx #velocity constraints
    ]
    
    return nothing
    
end

constraint! (generic function with 1 method)

In [8]:
#Boilerplate setup code to interface with IPOPT (don't touch this)

struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

function primal_bounds(n)
    x_l = -Inf*ones(n)
    x_u = Inf*ones(n)
    return x_l, x_u
end

function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)
    c_l[idx_ineq] .= -Inf

    c_u = zeros(m)
    return c_l, c_u
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    objective(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,objective,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    constraint!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), constraint!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function ipopt_solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-9,c_tol=1.0e-9,max_iter=10000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

ipopt_solve (generic function with 1 method)

In [9]:
#Solve with IPOPT
n_nlp = 2*n-1
m_nlp = 2+n
nlp_prob = ProblemMOI(n_nlp,m_nlp)

z_guess = zeros(2*n-1); #initial guess is all zeros
z_sol = ipopt_solve(z_guess,nlp_prob)

#We'll use the IPOPT solution as our reference for the following parts
y_ref = z_sol[1:n];
ẏ_ref = z_sol[(n+1):end];


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1323
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       49
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [10]:
#Plot Solution
plot(y_ref)

In [11]:
#Now we're going to implement the local solution technique based on the discrete Euler-Lagrange Equation

#One-step Discrete Lagrangian
function Gk(yk,yn,λ)
    
    ### Solution
    ym = 0.5*(yk+yn)
    ẏm = (yn-yk)/Δx
    (ρ*g*ym+λ)*sqrt(1 + ẏm^2)*Δx
end

#Discrete Euler-Lagrange Equation
function DEL(ym,yk,yn,λ)
    
    ### Solution
    ForwardDiff.derivative(dy->Gk(ym,dy,λ),yk) + ForwardDiff.derivative(dy->Gk(dy,yn,λ),yk)
end

DEL (generic function with 1 method)

In [12]:
#We need the Lagrange multiplier for the length constriant. We could get this out of IPOPT, but let's
#solve for it ourselves using the reference solution for y and the discrete Euler-Lagrange equation.

λ = 0.0

### Solution: There are several ways to do this knowing that the Lagrangian is always linear in the multiplier
r = DEL(y_ref[1],y_ref[2],y_ref[3],λ)
dr = ForwardDiff.derivative(dλ->DEL(y_ref[1],y_ref[2],y_ref[3],dλ), λ)
λ = -dr\r

52.367146716275165

In [13]:
#Now that we know λ, y[1], and y[2] from the reference solution, let's use the DEL to recover the rest of the solution y[3:n]

ytraj = zeros(n)
ytraj[1:2] = y_ref[1:2] #use initial conditions from reference solution
for k = 2:(n-1)
    
    ### Solution
    ytraj[k+1] = ytraj[k] #initial guess
    r = DEL(ytraj[k-1],ytraj[k],ytraj[k+1],λ) #residual
    while abs(r) > 1e-12
        dr = ForwardDiff.derivative(yn->DEL(ytraj[k-1],ytraj[k],yn,λ),ytraj[k+1])
        ytraj[k+1] = ytraj[k+1] - dr\r
        r = DEL(ytraj[k-1],ytraj[k],ytraj[k+1],λ)
    end
end

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref)
plot!(LinRange(0,w,n),ytraj)

In [14]:
#Check to make sure they really match
maximum(abs.(ytraj-y_ref))

6.524202765636884e-9

In [15]:
#Now let's implement the continuous ODE formulation of the problem.

#Continuous Lagrangian
function G(y,ẏ,λ)
    
    ### Solution
    (ρ*g*y+λ)*sqrt(1 + ẏ^2)
end

#Continuous Euler-Lagrange Equation
function EL(y,ẏ,λ)
    
    ### Solution
    DG = ForwardDiff.derivative(dy->G(dy,ẏ,λ),y)
    DDG = ForwardDiff.hessian(dx->G(dx[1],dx[2],λ),[y; ẏ])
    
    ÿ = DDG[2,2]\(DG - DDG[1,2]*ẏ)
end

EL (generic function with 1 method)

In [16]:
#Let's integrate the continuous version using DifferentialEquations.jl

#wrapper function for DifferentialEquations.jl interface 
function f(x,p,t)
    y = x[1]
    ẏ = x[2]
    ÿ = EL(y,ẏ,λ)
    return [ẏ; ÿ]
end

x0 = [y_ref[1]; ẏ_ref[1]]
tspan = (0.0,w)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5(),reltol=1e-8,abstol=1e-8);

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref)
plot!(sol, vars=1)

In [17]:
#Let's also integrate the continuous solution using implicit midpoint
xtraj1 = zeros(2,n)
xtraj1[:,1] .= x0

#Implicit midpoint step
function implicit_mid(xk)
    
    ### Solution
    xn = xk
    xm = 0.5*(xk + xn)
    r = xn - xk - Δx*f(xm,0,0)
    while maximum(abs.(r)) > 1e-12
        dr = I - 0.5*Δx*ForwardDiff.jacobian(dx->f(dx,0,0),xm)
        xn = xn - dr\r
        xm = 0.5*(xk + xn)
        r = xn - xk - Δx*f(xm,0,0)
    end
    
    return xn
end

#Simulate with implicit midpoint
for k = 1:(n-1)
    xtraj1[:,k+1] .= implicit_mid(xtraj1[:,k])
end

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref)
plot!(LinRange(0,w,n),xtraj1[1,:])

In [18]:
#Remember that ẏ[1] from our reference solution is a midpoint approximation, so it's actually ẏ(Δx/2), not ẏ(0).
#Let's try fixing this by taking a backwards step by Δx/2. You can use explicit Euler for this.

### Solution (using a half step of Euler - other methods are fine too)
ẏm = ẏ_ref[1]
ym = 0.5*(y_ref[1]+y_ref[2]) 
xm = [ym; ẏm]
x0 = xm - 0.5*Δx*f(xm,0,0)

2-element Vector{Float64}:
  0.0
 -2.667469488054186

In [19]:
#Let's try integrating with DifferentialEquations.jl again with this new x0
tspan = (0.0,w)
prob = ODEProblem(f,x0,tspan)
sol2 = solve(prob,Tsit5(),reltol=1e-8,abstol=1e-8);

#Compare against reference IPOPT solution
plot(LinRange(0,w,n),y_ref)
plot!(sol2, vars=1)

In [20]:
#Now let's try integrating with implicit midpoint again
xtraj2 = zeros(2,n)
xtraj2[:,1] = x0

#Simulate with implicit midpoint
for k = 1:(n-1)
    xtraj2[:,k+1] .= implicit_mid(xtraj2[:,k])
end

#Compare against reference IPOPT solution
plot(y_ref)
plot!(xtraj2[1,:])

In [21]:
#Check error vs. IPOPT reference solution
maximum(abs.(xtraj2[1,:]-y_ref))

6.5241069388612524e-9